In [ ]:
# known issue to fix: monster moved to a diagonal adjacent square and stopped even though it had more movement
# range was 1 but attack didn't hit
# i think this may have something to do with diagonal attacks not hitting

In [ ]:
''' 
    DND-like combat simulator
    This is a proof of concept - it works pretty well, but there are some known ugly bits, like the fact that 
    it exits with an error and that the input boxes float to the top sometimes
'''

In [ ]:
import random
from IPython.display import display, clear_output
import sys
import time

In [ ]:
# monsters are our actors
# they have core attributes (health, name, etc.) and a set of attacks they can do
# they will belong to a board, and they will send attacks out to the board to be carried out
class Monster:
    # basic monster setup
    def __init__(self, name, health, is_character, starting_location):
        self.name = name
        self.health = health
        # randomly generate a stack of possible attacks
        self.set_attacks()
        # is this the player character or the monster?
        self.is_character = is_character
        self.location = starting_location
        
    # think of this as a deck of attack cards that we will randomly pull from
    # here we generate that deck of attack cards
    def set_attacks(self):
        # each attack card will be generated with a strength, distance, and number of targets, so set 
        # some values to pull from
        strenghts = [1,2,3,4,5]
        strength_weights = [3,5,4,2,1]
        movements = [0,1,2,3,4]
        movement_weights = [1,3,4,3,1]
        max_distance = 3
        num_attacks = 5
        attacks = []
        
        # some things for attack names
        adjectives = ['Shadowed', 'Infernal', 'Venomous', 'Blazing', 'Cursed']
        elements = ['Fang', 'Storm', 'Flame', 'Void', 'Thorn']
        actions = ['Strike', 'Surge', 'Rend', 'Burst', 'Reaver']
        
        for item in [adjectives, elements, actions]:
            random.shuffle(item)

        # generate each attack card
        for i in range(num_attacks):
            strength = random.choices(strenghts, strength_weights)[0]
            movement = random.choices(movements, movement_weights)[0]
            distance = random.randint(1,max_distance)
            attack = {
                "attack_name": f"{adjectives.pop()} {elements.pop()} {actions.pop()}",
                "strength": strength,
                "distance": distance,
                "movement": movement
            }
            attacks.append(attack)
        self.attacks = attacks
    
    # grabs an attack card and send it to the board to be played
    def select_and_submit_attack(self, board):
        # grab a random attack card if it's the monster
        if self.is_character is False:
            attack_to_perform = random.choice(self.attacks)
            board.adjudicate_attack(attack_to_perform, self.is_character)
        
        # if it's the character, let them pick their own actions
        else:
            # if you run out of actions without killing the monster, you get exhausted
            if len(self.attacks)==0:
                print("Oh no! You're exhausted and have no more attacks left! Game over.")
                sys.exit()
            # if they have attacks, show them what they have
            print("Your attacks are: ")
            for i, attack in enumerate(self.attacks):
                print(f"{i}: {attack['attack_name']}: Strength {attack['strength']}, Distance: {attack['distance']}, Movement: {attack['movement']}")
            verifying = True
            # let them pick a valid attack
            while(verifying):
                attack_num = int(input("Which attack card would you like to pick? Type the number exactly."))
                if 0 <= attack_num < len(self.attacks):
                    verifying = False
                    clear_output()
                    attack_to_perform = self.attacks.pop(attack_num)
                else:
                    print("Oops, typo! Try typing the number again.")
            # send that attack to the board
            board.adjudicate_attack(attack_to_perform, self.is_character)

In [ ]:
# the board holds all the game metadata including the monster and character who are playing 
# it adjudicates actions and ends the game, too
# the board draws itself as well!
class Board:
    # set the game up by getting info from the player, giving instructions if needed, and start the turns
    # continue turns until the game is over!
    def __init__(self, size):
        self.size = size
        self.monster = Monster("Tree Man", 10, False, [2,3])
        player_name = input("What's your character's name? ")
        clear_output()
        self.character = Monster(player_name, 10, True, [0,0])
        self.game_over = False
        print("Welcome to your quest, "+player_name+". \nAs you enter the dungeon, you see a terrifying monster ahead! \nKill it or be killed...\n")
        want_help = input("Type anything to start or type help for instructions")
        if want_help == 'help':
            self.give_help()
        clear_output()
        self.draw()
        while self.game_over == False:
            self.take_turn()
    
    # dispaly the game instructions
    def give_help(self):
        clear_output()
        print('''
Welcome to the game! Here's how it works:
- You and the monster will attack each other once per turn in a random order
- You can only attack if you are within range of your enemy
- You pick your attacks, and the monster's are randomly generated
- Each attack has a movement associated with it. If you're not in range, you'll move that amount toward your enemy
- If you end in range, you will attack. If not, you won't attack this turn.
- Whoever runs out of health first loses

Good luck!
            ''')
        input("Type anything to continue")
        
    # draw the game board and display stats    
    def draw(self):
        print(f"Monster Health: {self.monster.health}, Character Health: {self.character.health}")
        to_draw = ''
        monster_square = False
        character_square = False
        for i in range(self.size):
            top = ' ---'*self.size+"\n"
            sides = ''
            for j in range(self.size+1):
                if self.character.location == [i,j]:
                    sides += '| C '
                elif self.monster.location == [i,j]:
                    sides += '| M '
                else:
                    sides += '|   '
            to_draw+=top
            to_draw+=sides+'\n'
        # add the bottom
        to_draw+=top
        print(to_draw)
        
    # is the attack in range? 
    def check_attack_in_range(self, attack_distance):
        return attack_distance >= sum(abs(a-b) for a,b in zip(self.monster.location, self.character.location))
    
    # carry out the attack
    def adjudicate_attack(self, attack, is_character):
        if self.game_over:
            print("Game over! Create a new board to try again")
            sys.exit(0)
        prefix = "Character " if is_character else "Monster "
        print(prefix+ "performs " + attack["attack_name"] + ": Attack "+ str(attack["strength"]) +", Range " +str(attack["distance"]) +", Movement " +str(attack["movement"]) + "\n")
        
        if is_character:
            self.perform_character_card(attack)
        else:
            self.perform_monster_card(attack)
        
        self.draw()
        time.sleep(3)
        
    def perform_character_card(self, attack):
        good_action = False
        self.draw()
        while not good_action:
            action = input("Type 1 to move first or 2 to attack first.")
            if action == "1":
                clear_output()
                good_action = True
                self.perform_character_movement(attack)
                clear_output()
                self.perform_attack(attack, True)
            elif action == "2":
                clear_output()
                good_action = True
                self.perform_attack(attack, True)
                clear_output()
                self.perform_character_movement(attack)
    
    def perform_character_movement(self, attack):
        print("Now it's time to move!")
        self.draw()
        movement_counter = attack["movement"]
        while movement_counter > 0:
            print(f"movement remaining: {movement_counter}")
            direction = input("Type w for up, a for left, d for right, s for down, or f to finish. If you move off the map, you'll disappear!")
            num_to_add = [0,0]
            if direction == "w":
                num_to_add = [-1,0]
            elif direction == "s":
                num_to_add = [1,0]
            elif direction == "a":
                num_to_add = [0,-1]
            elif direction == "d":
                num_to_add = [0,1]
            # would be nice to put something in here to catch moving off the map and to catch incorrect typing
            else:
                movement_counter = 0
            movement_counter -= 1
            self.character.location = [a+b for a,b in zip(self.character.location, num_to_add)]
            clear_output()
            self.draw()
        print("movement done!")                               
            
    
    def perform_monster_card(self, attack):
        if not self.check_attack_in_range(attack["distance"]):
            self.move(False, attack["movement"])
        
        # after movement, try to attack
        self.perform_attack(attack, False)
        
    def perform_attack(self, attack, is_character):
        # if you're close enough, attack
        if self.check_attack_in_range(attack["distance"]):
            print("Attack hits!\n")
            # if it's the character and the attack kills, end the game. Otherwise, increment monster health
            if is_character:
                if self.monster.health <= attack["strength"]:
                    self.win_game()
                else:
                    self.monster.health -= attack["strength"]
            # similar for monster
            else:
                if self.character.health <= attack["strength"]:
                    self.lose_game()
                else:
                    self.character.health -= attack["strength"]
        
        # if you're not close enough and already did your movement, that's it for your turn
        else:
            print("Not close enough to attack")

    def move(self, is_character, distance):
        y_dist = self.monster.location[0]-self.character.location[0]
        x_dist = self.monster.location[1]-self.character.location[1]
        
        # first move vertically if you can
        if distance > 0 and y_dist > 1:
            dist_to_travel = distance if (x_dist+y_dist) > distance else y_dist-1
            if is_character:
                self.character.location[0]+=dist_to_travel
            else:
                self.monster.location[0]-=dist_to_travel

            distance -= y_dist
            
        # if there's distance left, move horizontally
        if distance > 0 and x_dist > 1:
            dist_to_travel = distance if x_dist > distance else x_dist-1
            if is_character:
                self.character.location[1]+=dist_to_travel
            else:
                self.monster.location[1]-=dist_to_travel
                
    def take_turn(self):
        # randomize who starts the turn
        if random.choice([1,2]) == 1:
            # monster goes first
            clear_output()
            print("Ah! The Monster got a jump on you this round. Watch out...")
            self.monster.select_and_submit_attack(self)
            print("Now it's your turn! Make this one count.")
            self.character.select_and_submit_attack(self)
            input("Type anything to start the next round")
        else:
            # character goes first
            clear_output()
            print("You acted quick - it's your turn first this time.")
            self.draw()
            self.character.select_and_submit_attack(self)
            print("Now it's the Monster's turn...Watch out!")
            self.monster.select_and_submit_attack(self)
            input("Type anything to start the next round")
            
            
    def lose_game(self):
        print('''You died...GAME OVER
  .-.
 (o o)  
  |-|  
 /   \\
|     |
 \\___/''')
        self.game_over = True
        self.character.location = (self.size+1, self.size+1)
        sys.exit(0)
        
    def win_game(self):
        clear_output()
        print("You defeated the monster!!")
        self.game_over = True
        print('''
    \o/   Victory!
     |
    / \
   /   \
        ''')
        self.monster.location = (self.size+1, self.size+1)
        sys.exit(0)

In [ ]:
# this line starts the game - you can change board size if you want!
board = Board(5)